Import neccessary dependencies and api urls

In [41]:
import pandas as pd
import seaborn as sb
import requests as rq
from urls import SILVER_PRODUCTION_URL

In [42]:
def get_data(url: str = "", path: str = "") -> pd.DataFrame:

    entries_df = pd.DataFrame(
        columns = ["TIME_PERIOD", "OBS_VALUE", "COBER_GEO"]
    )

    # Data retrieval when a url is given
    if url != "":
        r = rq.get(url)
        data = r.json()['Series'][0]['OBSERVATIONS']

        for entry_no in range(len(data)):
            entries_list = [
                data[entry_no]["TIME_PERIOD"],
                data[entry_no]["OBS_VALUE"],
                data[entry_no]["COBER_GEO"],
            ]

            entries_df.loc[len(entries_df)] = entries_list

    return entries_df

In [43]:
data = get_data(SILVER_PRODUCTION_URL)
print(data)

    TIME_PERIOD OBS_VALUE COBER_GEO
0       2025/08    334299        00
1       2025/07    323844        00
2       2025/06    322758        00
3       2025/05    341994        00
4       2025/04    350039        00
..          ...       ...       ...
543     1980/05    112018        00
544     1980/04    120735        00
545     1980/03    135648        00
546     1980/02    123887        00
547     1980/01    115411        00

[548 rows x 3 columns]
